<a href="https://colab.research.google.com/github/Elwing-Chou/ml0223/blob/main/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets.mnist import load_data
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
layers = [
    Dense(64, activation="relu", input_dim=784),
    Dropout(0.25),
    Dense(1, activation="sigmoid")
]
dis = Sequential(layers)
dis.summary()

In [ ]:
from tensorflow.keras.layers import BatchNormalization
layers = [
    Dense(256, activation="relu", input_dim=100),
    BatchNormalization(),
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dense(784, activation="tanh")
]
gen = Sequential(layers)
gen.summary()

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy
dis.compile(loss=BinaryCrossentropy(),
            optimizer="adam")

In [ ]:
from tensorflow.keras.models import Model
dis.trainable = False
out = dis(gen.output)
gan = Model(gen.input, out)
gan.summary()

In [ ]:
gan.compile(loss=BinaryCrossentropy(),
            optimizer="adam")

In [ ]:
x_train_norm = (x_train.reshape(60000, 784) - 127.5) / 127.5

In [ ]:
import numpy as np
valid = np.ones((20, )) 
fake = np.zeros((20, ))

In [ ]:
for i in range(100000):
    # dis
    idx = np.random.randint(0, 60000, size=20)
    img = x_train_norm[idx]
    r1 = dis.train_on_batch(img, valid)
    rand = np.random.normal(size=(20, 100))
    imgf = gen.predict(rand)
    r2 = dis.train_on_batch(imgf, fake)
    if i % 100 == 0:
        print("dis loss:", (r1 + r2) / 2)
    # gen
    rand = np.random.normal(size=(20, 100))
    r3 = gan.train_on_batch(rand, valid)
    if i % 100 == 0:
        print("gen loss:", r3)

In [ ]:
import matplotlib.pyplot as plt
rand = np.random.normal(size=(5, 100))
ans = (gen.predict(rand).reshape(5, 28, 28) + 1) / 2
plt.figure(figsize=(10, 2))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.axis("off")
    plt.imshow(ans[i], cmap="gray")